In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [42]:
df = pd.read_csv('loan.csv', low_memory=False)
dcoldf = pd.DataFrame(df.columns.to_list(), columns=['LoanStatNew'])
dcoldf.shape

(111, 1)

In [53]:
dcoldf['bar'] = 5

In [44]:
dd = pd.read_excel('dd.xlsx', sheet_name='LoanStats')
dd.shape

(127, 4)

In [49]:
dd = dd[~dd['LoanStatNew'].isna()]
dd.shape

(115, 4)

In [56]:
foo = pd.merge(dd, dcoldf, on='LoanStatNew', how='left')
foo[foo['bar'].isna()]

,LoanStatNew,Description,categorical/numerical,My Understanding,bar
33,fico_range_high,The upper boundary range the borrower’s FICO a...,NaN,NaN,NaN
34,fico_range_low,The lower boundary range the borrower’s FICO a...,NaN,NaN,NaN
42,last_fico_range_high,The upper boundary range the borrower’s last F...,NaN,NaN,NaN
43,last_fico_range_low,The lower boundary range the borrower’s last F...,NaN,NaN,NaN
111,total_rev_hi_lim,Total revolving high credit/credit limit,NaN,NaN,NaN
113,verified_status_joint,Indicates if the co-borrowers' joint income wa...,NaN,NaN,NaN


In [63]:
df['verified_status_joint'].info()

KeyError: 'verified_status_joint'

In [65]:
foo = pd.merge(dd, dcoldf, on='LoanStatNew', how='left')
bar = foo[['LoanStatNew', 'Description']]
bar.to_excel('temp.xlsx')

In [84]:
col_condition = df.isna().sum() > 35000
columns_to_drop = df.columns[col_condition].to_list()
df.drop(columns=columns_to_drop, inplace=True)

In [119]:
df.drop(columns=['mths_since_last_delinq'], inplace=True)

In [212]:
column = 'collections_12_mths_ex_med'
print(df[column].info())
print(df[column].unique().size)

<class 'pandas.core.series.Series'>
RangeIndex: 39717 entries, 0 to 39716
Series name: collections_12_mths_ex_med
Non-Null Count  Dtype  
--------------  -----  
39661 non-null  float64
dtypes: float64(1)
memory usage: 310.4 KB
None
2


In [213]:
df['collections_12_mths_ex_med'].unique()

array([ 0., nan])

In [181]:
df.head()[[
    'member_id',
    'loan_amnt',
    'last_pymnt_amnt',
    'last_pymnt_d',
    'out_prncp',
    'total_pymnt',
    'total_pymnt_inv'
]]

,member_id,loan_amnt,last_pymnt_amnt,last_pymnt_d,out_prncp,total_pymnt,total_pymnt_inv
0,1296599,5000,171.62,Jan-15,0.00,5863.155187,5833.84
1,1314167,2500,119.66,Apr-13,0.00,1008.710000,1008.71
2,1313524,2400,649.91,Jun-14,0.00,3005.666844,3005.67
3,1277178,10000,357.48,Jan-15,0.00,12231.890000,12231.89
4,1311748,3000,67.79,May-16,524.06,3513.330000,3513.33


In [214]:
df.iloc[0,:]

id                                                                      1077501
member_id                                                               1296599
loan_amnt                                                                  5000
funded_amnt                                                                5000
funded_amnt_inv                                                          4975.0
term                                                                  36 months
int_rate                                                                 10.65%
installment                                                              162.87
grade                                                                         B
sub_grade                                                                    B2
emp_title                                                                   NaN
emp_length                                                            10+ years
home_ownership                          